1\. Write a function that converts number representation (bin<->dec<->hex)

In [29]:
#decimal to binary
def dec_bin(x):
    result=""
    while x>=1:
        result=result+str(x%2)
        x//=2
    print("decimal to binary:",int(result[::-1]))

#binary to decimal
def bin_dec(x):
    a=[]
    result=0
    a[:0]=x  
    a=[int(i) for i in a] #convert the list of strings into a list of int
    for i in range(len(a)):
        digit=a.pop()
        if digit==1:
            result=result+pow(2,i)
    print("binary to decimal:",result)
    
#decimal to hexadecimal
def dec_hex(x):
    result=''
    conv=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A' , 'B', 'C', 'D', 'E', 'F']
    while x>0:
        a=x%16
        result=conv[a]+result
        x=x//16
    print("decimal to hexadecimal:",result)   

#hexadecimal to decimal
def hex_dec(x):
    result=0
    conv={'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10 , 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15}
    a=len(x)-1
    for i in x:
        result +=conv[i]*pow(16,a)
        a -=1
    print("hexadecimal to decimal:", result)
        
dec_bin(44)
bin_dec("101100")
dec_hex(44)
hex_dec("2C")       

decimal to binary: 101100
binary to decimal: 44
decimal to hexadecimal: 2C
hexadecimal to decimal: 44


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [66]:
def converter(x):
    sign_bit=x[0]
    exp_bits=x[1:9]
    mant_bits=x[9:]
    
    #sign
    sign=pow(-1,int(sign_bit))
    
    #convert exponent
    temp=[]
    exp_res=0
    temp[:0]=exp_bits
    temp=[int(i) for i in temp]
    for i in range(len(temp)):
        digit=temp.pop()
        if digit==1:
            exp_res=exp_res+pow(2,i)
    exponent=pow(2,(exp_res-127))
        
    #convert mantissa
    temp=[]
    mant_res=0
    temp[:0]=mant_bits
    temp=[int(i) for i in temp]
    temp=temp[::-1]
    for i in range(len(temp)):
        digit=temp.pop()
        if digit==1:
            mant_res=mant_res+pow(2,-(i+1))
    mantissa=1+mant_res
    print(sign*exponent*mantissa)
    
    
converter("10000011111000000000000000000000")

-1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [22]:
def under(x):
    l=[]
    while x!=0.0:
        x/=2
        if x!=0.0: 
            l.append(x)
    print("underflow:",l[-1])
        
def over(x):
    l=[]
    while x<float("inf"):
        x*=2
        l.append(x)
    print("overflow:",l[-2])
        
    
a=1.0
under(a)
over(a)

underflow: 5e-324
overflow: 8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [41]:
def epsilon(a,b):
    l=[]
    while b!=0.0:
        b/=2
        a=a+b
        if a!=2.0:
            l.append(a)
    print("machine precision:",2-(l[-1]))
    
a=1.0
b=1.0
epsilon(a,b)


machine precision: 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [26]:
def sol1(a,b,c):
    delta=pow(pow(b,2)-(4*a*c),0.5)
    x1=(-b+delta)/(2*a)
    x2=(-b-delta)/(2*a)
    return x1,x2

def sol2(a,b,c):
    delta=pow(pow(b,2)-(4*a*c),0.5)
    x1=(2*c)/(-b-delta)
    x2=(2*c)/(-b+delta)
    return x1,x2

def sol3(a,b,c):
    delta=pow(pow(b,2)-(4*a*c),0.5)
    x1=(-b+delta)/(2*a)
    x2=(2*c)/(-b+delta)
    return x1,x2

a=0.001
b=1000
c=0.001

print("method 1")
print("x1, x2:",sol1(a,b,c))
print("method 2")
print("x1, x2:",sol2(a,b,c))
print("method 1+2")
print("x1, x2:",sol3(a,b,c))

method 1
x1, x2: (-9.999894245993346e-07, -999999.999999)
method 2
x1, x2: (-1.000000000001e-06, -1000010.5755125057)
method 1+2
x1, x2: (-9.999894245993346e-07, -1000010.5755125057)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [60]:
def derivative(x,y,f):
    d=pow(10,-y)
    return (f(x+d)-f(x))/d
    
def func(x):
    return x*(x-1)
    
for y in range(2,15,2):
    print("d=10^(-{}):".format(y),derivative(1,y,func),)

d=10^(-2): 1.010000000000001
d=10^(-4): 1.0000999999998899
d=10^(-6): 1.0000009999177333
d=10^(-8): 1.0000000039225287
d=10^(-10): 1.000000082840371
d=10^(-12): 1.0000889005833413
d=10^(-14): 0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [25]:
import numpy as n

def riemann(n):
    h=2/n
    a=0.0
    for i in range(0,n+1):
        a=a+(h*pow(1-(i/n)**2,0.5))
    return a

print("true value:",n.pi/2)
print("riemann value for n=100:",riemann(100))
print("riemann value for n=100000:",riemann(100000))
print("riemann value for n=1000000:",riemann(1000000))
print("riemann value for n=10000000:",riemann(10000000))
print("riemann value for n=100000000:",riemann(100000000))


true value: 1.5707963267948966
riemann value for n=100: 1.5802085158895236
riemann value for n=100000: 1.5708063082009784
riemann value for n=1000000: 1.5707973262069104
riemann value for n=10000000: 1.5707964267761794
riemann value for n=100000000: 1.5707963367946078
